# LeakyReLU — Matematiksel Temel, Türev, Backprop ve Pratik Sonuçlar

Bu defter, **LeakyReLU** aktivasyonunun matematiğini uçtan uca ele alır:

- Tanım (piecewise)
- Türev / subgradient ve 0 noktasındaki durum
- Backprop’ta gradyan akışı: “buradan dolayı şu oldu”
- ReLU ile farkın teorik sonucu: *dying ReLU* ve gradyan sönmesi
- Varyans/ölçek etkisi (init ve norm ile ilişkisi)
- α (negative_slope) seçiminin etkileri
- PReLU bağlantısı
- Kısa doğrulama kodları (minimal)

> Not: Matematik var, ama hedef “formül ezberi” değil; sonuçları **neden-sonuç** ilişkisiyle kurmak.

## 1) Tanım

LeakyReLU, negatif bölgede küçük bir eğim (sızıntı) bırakan piecewise lineer fonksiyondur.

Bir parametre seçilir:  
- \( \alpha \in (0, 1) \) genelde küçük (örn. 0.01 veya 0.1)

Fonksiyon:

\[
\mathrm{LeakyReLU}_\alpha(x) =
\begin{cases}
x, & x \ge 0 \\
\alpha x, & x < 0
\end{cases}
\]

**ReLU** bunun özel halidir: \( \alpha = 0 \).

## 2) Süreklilik ve türev (genel)

LeakyReLU fonksiyonu **her noktada süreklidir**.  
Özellikle \( x = 0 \) noktasında:

- Sol limit: \( \lim_{x \to 0^-} \alpha x = 0 \)
- Sağ limit: \( \lim_{x \to 0^+} x = 0 \)

İki parça da aynı değeri verdiği için fonksiyon **0 noktasında süreklidir**.


Türev, \( x = 0 \) **hariç** olmak üzere açıkça tanımlıdır:

\[
\frac{d}{dx}\mathrm{LeakyReLU}_\alpha(x) =
\begin{cases}
1, & x > 0 \\
\alpha, & x < 0
\end{cases}
\]

### 0 noktasında ne olur?

\( x = 0 \) noktasında sol türev \( \alpha \), sağ türev ise \( 1 \)’dir.  
Bu nedenle klasik anlamda **tek bir türev değeri yoktur**.

Derin öğrenme pratiğinde bu durum problem oluşturmaz; çünkü:
- \( x = 0 \) noktası **ölçüsü sıfır** olan bir kümedir
- Sürekli dağılımlardan örneklenen verilerde tam olarak bu noktaya düşme olasılığı ihmal edilebilirdir

Bu yüzden otomatik türev alan framework’ler (PyTorch, TensorFlow vb.)  
\( x = 0 \) için **subgradient** yaklaşımı kullanır ve genellikle:
- \( 1 \),
- \( \alpha \),
- veya framework’e özgü deterministik bir değeri

atanır. Bu seçim, eğitim sürecini pratikte etkilemez.
``


## 3) Backprop: “Buradan dolayı bu oldu”

Bir katmanda ileri yayılım şu şekildedir:

- Ön-aktivasyon (pre-activation):  
  \[
  z = W x + b
  \]

- Aktivasyon:  
  \[
  a = f(z), \quad \text{burada } f = \mathrm{LeakyReLU}_\alpha
  \]

Kayıp fonksiyonu aktivasyon üzerinden tanımlıdır:
\[
\mathcal{L} = \mathcal{L}(a)
\]



### Zincir kuralı

Zincir kuralına göre, kaybın \( z \)’ye göre türevi:

\[
\frac{\partial \mathcal{L}}{\partial z}
=
\frac{\partial \mathcal{L}}{\partial a}
\cdot
\frac{da}{dz}
=
\delta \cdot f'(z)
\]

burada
\[
\delta = \frac{\partial \mathcal{L}}{\partial a}
\]
olarak tanımlanır.


### LeakyReLU için türev davranışı

LeakyReLU’nun türevi parça parça tanımlıdır:

- Eğer \( z > 0 \) ise:
  \[
  f'(z) = 1
  \quad \Rightarrow \quad
  \frac{\partial \mathcal{L}}{\partial z} = \delta
  \]

- Eğer \( z < 0 \) ise:
  \[
  f'(z) = \alpha
  \quad \Rightarrow \quad
  \frac{\partial \mathcal{L}}{\partial z} = \alpha \, \delta
  \]


### Sonuç (neden–sonuç ilişkisi)

- **ReLU**’da negatif bölgede \( f'(z) = 0 \) olduğu için:
  \[
  \frac{\partial \mathcal{L}}{\partial z} = 0
  \]
  Bu durumda negatif tarafta **gradyan akışı tamamen kesilir**.

- **LeakyReLU**’da ise negatif bölgede \( f'(z) = \alpha > 0 \) olduğundan:
  \[
  \frac{\partial \mathcal{L}}{\partial z} = \alpha \, \delta \neq 0
  \]

  Bu nedenle gradyan akışı **tamamen kaybolmaz**.  
  Nöron negatif bölgede kalsa bile öğrenmeye devam eder; yalnızca gradyan **\( \alpha \) katsayısı kadar zayıflar**.


## 4) “Dying ReLU” matematiksel bakış

ReLU’da bir nöronun pre-activation’ı uzun süre negatifte kalırsa:

- Forward: çıktısı 0’a kilitlenir  
- Backward: türev 0 olduğu için ağırlık gradyanı 0 olur

Örneğin tek nöronlu durumda:
\[
z = w^\top x + b,\quad a=\max(0,z)
\]

\[
\frac{\partial \mathcal{L}}{\partial w}
=
\frac{\partial \mathcal{L}}{\partial a}
\cdot
\frac{da}{dz}
\cdot
\frac{\partial z}{\partial w}
=
\delta \cdot \mathbf{1}[z>0] \cdot x
\]

Eğer \(z<0\) ise \(\mathbf{1}[z>0]=0\) ve gradyan **tam 0**.  
Bu, nöronun ağırlıklarının “negatif taraftan geri dönmesini” zorlaştırır.

LeakyReLU’da aynı ifade:

\[
\frac{\partial \mathcal{L}}{\partial w}
=
\delta \cdot
\begin{cases}
1, & z>0 \\
\alpha, & z<0
\end{cases}
\cdot x
\]

Buradan dolayı:
- \(z<0\) iken bile \(\alpha \delta x\) gradyanı gelir  
- nöron parametreleri güncellenir  
- zamanla \(z\) dağılımı pozitife kayabilir (veya daha dengeli hale gelebilir)

**Özet:** ReLU’da “negatifte kalırsan öğrenemezsin”; LeakyReLU’da “negatifte kalırsan daha yavaş öğrenirsin”.

## 5) Ölçek/variance etkisi (init ve dağılım)

LeakyReLU lineer parçalardan oluştuğu için giriş dağılımının ölçeğini etkiler.

Varsayım (yaklaşık analiz):
- \(z\) simetrik bir dağılımdan geliyor (örn. ortalaması 0)

LeakyReLU çıktısı:
- pozitif yarıda \(z\)
- negatif yarıda \(\alpha z\)

Bu durumda ikinci moment (kabaca):

\[
\mathbb{E}[f(z)^2]
=
\mathbb{E}[z^2\mathbf{1}[z>0]] + \mathbb{E}[\alpha^2 z^2\mathbf{1}[z<0]]
\]

Simetriden dolayı:

\[
\mathbb{E}[z^2\mathbf{1}[z>0]] \approx \frac{1}{2}\mathbb{E}[z^2],\quad
\mathbb{E}[z^2\mathbf{1}[z<0]] \approx \frac{1}{2}\mathbb{E}[z^2]
\]

Dolayısıyla:

\[
\mathbb{E}[f(z)^2]
\approx
\frac{1+\alpha^2}{2}\mathbb{E}[z^2]
\]

### Buradan çıkan pratik sonuç

- ReLU için \(\alpha=0\): \(\frac{1+\alpha^2}{2}=\frac{1}{2}\)  
  ➜ aktivasyon ikinci momenti düşürür (ölçek küçülür)
- LeakyReLU için \(\alpha>0\): \(\frac{1+\alpha^2}{2}\) daha büyük  
  ➜ ölçek “ReLU kadar” düşmeyebilir

Bu, özellikle **He/Kaiming init** gibi yöntemlerin neden aktivasyon tipine göre ayarlandığını açıklar.

## 6) Kaiming/He init ile ilişki (neden α önemli)

Kaiming init’in amacı: katmanlar arası sinyalin (ve gradyanın) **patlamadan/sönmeden** taşınması.

LeakyReLU için Kaiming init genelde “a=α” parametresiyle kullanılır.  
Çünkü aktivasyonun ikinci momenti ve efektif kazancı α’ya bağlıdır.

PyTorch tarafında bu ilişki şuna karşılık gelir:
- `nn.init.kaiming_normal_(..., a=negative_slope, nonlinearity="leaky_relu")`

**Buradan dolayı:** α değiştirirsen, init de “ideal” noktadan sapabilir.  
(Pratikte BN/GN varsa bu etki azalır ama tamamen yok olmaz.)

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init

w = torch.empty(128, 128)  # örnek ağırlık matrisi
alpha = 0.1

init.kaiming_normal_(w, a=alpha, nonlinearity="leaky_relu")
print(w.mean().item(), w.std().item())

-0.0010277541587129235 0.12382026761770248


## 7) Lipschitz / gradyan üst sınırı (stabilite açısından)

LeakyReLU piecewise lineer ve türevi \(\in \{\alpha, 1\}\).

Bu, fonksiyonun Lipschitz sabitinin (en büyük türev büyüklüğü) yaklaşık **1** olmasını sağlar:

\[
|f(x_1)-f(x_2)| \le 1\cdot|x_1-x_2|
\]

**Buradan dolayı:**
- Aktivasyon tek başına gradyanı büyütmez (maksimum çarpan 1)
- Negatif tarafta gradyanı küçültür (çarpan α)
- Bu da bazı eğitim senaryolarında daha “kontrollü” bir gradyan akışı sağlar

## 8) α (negative_slope) seçimi — teorik/pratik trade-off

Negatif tarafta gradyan katsayısı α:

- α küçükse (0.01):
  - ReLU’ya çok benzer
  - Dying ReLU riskini azaltır ama negatifte öğrenme yavaş

- α büyükse (0.1, 0.2):
  - Negatifte daha çok bilgi/gradient taşınır
  - “Daha lineer” davranış artar (çok büyürse ReLU benzeri keskin ayrım azalır)

**Buradan dolayı:** α büyütmek her zaman iyi değildir; kapasite/regularization dengesi değişir.

Pratikte object detection’da 0.1 sık görülür; CNN genelinde 0.01 yaygındır.

## 9) LeakyReLU ↔ PReLU bağı

PReLU şu fikri getirir:
- α sabit olmasın, öğrenilsin.

PReLU tanımı:

\[
\mathrm{PReLU}(x) =
\begin{cases}
x, & x \ge 0 \\
a x, & x < 0
\end{cases}
\quad\text{burada } a \text{ öğrenilir}
\]

**Buradan dolayı:**
- Model negatif taraftaki “ne kadar sızdıracağını” veriye göre ayarlar
- Ama:
  - ekstra parametre
  - overfit riski
  - bazen küçük fayda / ek karmaşıklık

LeakyReLU daha “sabit ve ucuz” bir çözümdür.

## 10) BatchNorm / GroupNorm ile birlikte neden etkisi bazen azalır?

Norm katmanları (BN/GN), aktivasyon öncesi dağılımı düzenler:
- ortalamayı/ölçeği stabilize eder
- z değerlerinin sürekli aşırı negatifte kalmasını azaltabilir

**Buradan dolayı:**
- ReLU’nun dying riski bazı ağlarda zaten düşük kalabilir
- LeakyReLU farkı küçülebilir

Ama şu durumlarda LeakyReLU farkı tekrar görünür:
- small batch (BN’nin zayıflaması)
- domain shift
- aşırı LR
- agresif regularization / pruning

## 11) Backprop’ta bir “gradyan kapısı” olarak LeakyReLU

Aktivasyonları “gate” gibi düşünürsen:

- ReLU: negatifte kapı **tam kapanır**
- LeakyReLU: negatifte kapı **kısmen açık kalır**

\[
\frac{\partial \mathcal{L}}{\partial z} = \delta \cdot g(z),
\quad
g(z)\in\{\alpha,1\}
\]

**Buradan dolayı:**
- gradient flow daha sürekli hale gelir
- bazı katmanlarda “öğrenme durması” daha az olur

## 12) Minimal doğrulama: türev davranışı (numerik kontrol)

Aşağıdaki hücre:
- autograd türevi ile
- numerik fark türevini
yaklaştırmalı karşılaştırır.

(0 noktasına çok yaklaşınca parça geçişi nedeniyle dalgalanma normaldir.)

In [2]:
import torch
import torch.nn.functional as F

def num_grad(f, x, eps=1e-4):
    return (f(x + eps) - f(x - eps)) / (2 * eps)

alpha = 0.1
x = torch.tensor([-1.0, -0.2, -1e-6, 0.0, 1e-6, 0.2, 1.0], requires_grad=True)

y = F.leaky_relu(x, negative_slope=alpha).sum()
y.backward()
auto = x.grad.detach().clone()

x2 = x.detach()
ng = num_grad(lambda t: F.leaky_relu(t, negative_slope=alpha).sum(), x2)

print("x:", x2.tolist())
print("autograd:", auto.tolist())
print("numgrad :", ng.tolist())

x: [-1.0, -0.20000000298023224, -9.999999974752427e-07, 0.0, 9.999999974752427e-07, 0.20000000298023224, 1.0]
autograd: [0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 1.0, 1.0, 1.0]
numgrad : 3.8504600524902344


## 13) “Dying” etkisini oyuncak bir deneyle sezmek (mini)

Aşağıdaki mini deney:
- Rastgele lineer katman
- ReLU vs LeakyReLU
- Negatif bölgede kalan oranı ve gradyanların sıfırlanma oranını kaba şekilde gözler

Bu, gerçek ağların yerine geçmez; sadece mekanizmayı görselleştirir.

In [3]:
import torch
import torch.nn as nn

torch.manual_seed(0)

N, D = 4096, 64
x = torch.randn(N, D)

lin = nn.Linear(D, D, bias=True)

z = lin(x)  # pre-activation

relu = nn.ReLU()
lrelu = nn.LeakyReLU(0.1)

a_relu = relu(z)
a_lrelu = lrelu(z)

neg_ratio = (z < 0).float().mean().item()
zero_ratio_relu = (a_relu == 0).float().mean().item()

# "gradyan kapısı" oranını sezmek için: türev maskesi
mask_relu = (z > 0).float()
mask_lrelu = torch.where(z > 0, torch.ones_like(z), torch.full_like(z, 0.1))

dead_grad_ratio_relu = (mask_relu == 0).float().mean().item()
dead_grad_ratio_lrelu = (mask_lrelu == 0).float().mean().item()

print(f"Negatif oran (z<0): {neg_ratio:.3f}")
print(f"ReLU çıktı 0 oranı: {zero_ratio_relu:.3f}")
print(f"ReLU'da gradyan tamamen kapanan oran: {dead_grad_ratio_relu:.3f}")
print(f"LeakyReLU'da gradyan tamamen kapanan oran: {dead_grad_ratio_lrelu:.3f}")

Negatif oran (z<0): 0.499
ReLU çıktı 0 oranı: 0.499
ReLU'da gradyan tamamen kapanan oran: 0.499
LeakyReLU'da gradyan tamamen kapanan oran: 0.000


## 14) Sonuçların özeti (maddeler halinde)

- LeakyReLU, ReLU’nun “negatifte gradyanı sıfırlama” davranışını yumuşatır.
- Türev negatifte α olduğu için gradyan akışı **kesilmez**, sadece ölçeklenir.
- Bu, “dying ReLU” riskini teorik olarak düşürür:
  - ReLU: negatifte tam kapı (0)
  - LeakyReLU: negatifte kısmi kapı (α)
- α büyüdükçe negatif bilgi taşınır; ama fonksiyon daha lineerleşir (trade-off).
- Init (Kaiming/He) ve aktivasyon kazancı α’ya bağlıdır; α değişirse ideal init ayarı da değişir.
- BN/GN gibi normlar varsa fark bazen küçülür, ama small-batch / domain shift gibi durumlarda tekrar değer kazanabilir.

## 15) “Matematik defteri” sonrası pratik öneri

Bu matematik kısmından çıkan pratik aksiyon:
1. LeakyReLU kullanıyorsan α’yı sabitle (0.01 veya 0.1)  
2. Kaiming init’de `a=α` kullan (eğer init’i manuel yapıyorsan)  
3. Ablation ile (tek değişken aktivasyon) mAP + stability + latency ölç  
4. Eğer model zaten SiLU ile güçlü baseline veriyorsa, LeakyReLU’yu “stabilite/hız” için tercih et